In [1]:
import pandas as pd
import numpy as np

import datetime as dt

In [2]:
from IPython.core.display import display, HTML # Расширить рабочее поле ноутбука на весь экран
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
pd.options.display.max_colwidth = 300 # Увеличить длинну отображаемых строк

# Генерация данных

In [4]:
data = pd.DataFrame([
            ['16-10-2021', '23:56:10', 'A', 'a1', '0001'], 
            ['16-10-2021', '23:58:30', 'A', 'a1', '0002'],
            ['16-10-2021', '23:58:35', 'B', 'b1', '0003'],
            ['16-10-2021', '23:58:56', 'B', 'b2', '0004'],
            ['16-10-2021', '23:59:21', 'C', 'c1', '0005'],
            ['16-10-2021', '23:59:42', 'C', 'c1', '0006'],

            ['17-10-2021', '00:03:05', 'A', 'a1', '0007'],
            ['17-10-2021', '00:03:25', 'A', 'a2', '0008'],
            ['17-10-2021', '00:03:48', 'A', 'a3', '0009'],
            ['17-10-2021', '00:05:24', 'B', 'b1', '0010'],
            ['17-10-2021', '00:05:36', 'B', 'b1', '0011'],
            ['17-10-2021', '00:05:44', 'B', 'b2', '0012'],
            ['17-10-2021', '00:06:03', 'B', 'b2', '0013'],
            ['17-10-2021', '03:12:13', 'B', 'b3', '0014'],
            ['17-10-2021', '03:12:46', 'C', 'c1', '0015'],
            ['17-10-2021', '13:06:54', 'C', 'c1', '0016'], 
            ['17-10-2021', '13:12:10', 'C', 'c2', '0017'], 
            ['17-10-2021', '19:48:30', 'C', 'c2', '0018'], 

            ['18-10-2021', '00:06:05', 'C', 'c1', '0019'], 
            ['18-10-2021', '03:08:03', 'C', 'c2', '0020'],
            ['18-10-2021', '12:07:08', 'C', 'c3', '0021'],
    
            ['19-10-2021', '00:04:03', 'A', 'a1', '0022'], 
            ['19-10-2021', '02:08:20', 'A', 'a2', '0023'],
            ['19-10-2021', '10:03:08', 'B', 'a2', '0024'],
            ['19-10-2021', '12:07:08', 'B', 'b1', '0025'],
    
], columns=['str_date', 'str_time','group', 'sub_group','val_id'])

data['datetime'] = data.apply(lambda row:  dt.datetime.strptime(f'{row.str_date} {row.str_time}', '%d-%m-%Y %H:%M:%S'), axis=1)

data['date'] = data.datetime.dt.date
data['time'] = data.datetime.dt.time
data['week'] = data.datetime.dt.week # Получить номер недели

# data['weekday'] = data.datetime.dt.strftime('%A') 
data['weekday'] = data.datetime.dt.day_name() # Получить день недели (на английском)

datetime_cols = ['datetime', 'date','time','week','weekday']
group_cols = ['group', 'sub_group']
data = data[[*datetime_cols, *group_cols, 'val_id']]

data.head()

/DataScience/safonkin/PycharmProjects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,datetime,date,time,week,weekday,group,sub_group,val_id
0,2021-10-16 23:56:10,2021-10-16,23:56:10,41,Saturday,A,a1,0001
1,2021-10-16 23:58:30,2021-10-16,23:58:30,41,Saturday,A,a1,0002
2,2021-10-16 23:58:35,2021-10-16,23:58:35,41,Saturday,B,b1,0003
3,2021-10-16 23:58:56,2021-10-16,23:58:56,41,Saturday,B,b2,0004
4,2021-10-16 23:59:21,2021-10-16,23:59:21,41,Saturday,C,c1,0005


# Таблица

In [5]:
table = (
    data
        .pivot_table(
                    index=['date', 'weekday'], 
                    columns=['group','sub_group'], 
                    values='val_id', 
                    aggfunc='nunique'
        )
        .fillna(0)
)
table

group                  A              B                   C          
sub_group             a1   a2   a3   a2   b1   b2   b3   c1   c2   c3
date       weekday                                                   
2021-10-16 Saturday  2.0  0.0  0.0  0.0  1.0  1.0  0.0  2.0  0.0  0.0
2021-10-17 Sunday    1.0  1.0  1.0  0.0  2.0  2.0  1.0  2.0  2.0  0.0
2021-10-18 Monday    0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
2021-10-19 Tuesday   1.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0

In [6]:
print(table.columns)
A_cols = [col for col in table.columns if 'A' in col]
B_cols = [col for col in table.columns if 'B' in col]
C_cols = [col for col in table.columns if 'C' in col]

MultiIndex([('A', 'a1'),
            ('A', 'a2'),
            ('A', 'a3'),
            ('B', 'a2'),
            ('B', 'b1'),
            ('B', 'b2'),
            ('B', 'b3'),
            ('C', 'c1'),
            ('C', 'c2'),
            ('C', 'c3')],
           names=['group', 'sub_group'])


In [7]:
table_styled = (
    table.style

            .set_properties(**{
                'text-align': 'center', 
                'width': '200px'})
            .format({col: "{}" for col in table.columns})
            .background_gradient(subset=A_cols, cmap='Greens')
            .background_gradient(subset=B_cols, cmap='Greys')
            .background_gradient(subset=C_cols, cmap='Reds')
            .set_table_styles([{'selector': 'th', 'props': [('border-color', 'black'), 
                                                            ('border-style','solid')]}])
    
)

table_styled